In [ ]:
from __future__ import division

from Queue import Queue, LifoQueue

import numpy as np

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=True)

With feedback
-------------

In the steady state, we have the following traffic equation.
$$ \lambda_{ext} + (1 - \mathbb{E}[e^{-\theta D}])\lambda = \lambda < \mu $$  

Assuming a FIFO service discipline, we can approximate delay as $D \sim Exponential(\mu - \lambda)$. Noticing that the expected recall rate is the moment-generating function for $D$,

$$ \mathbb{E}[e^{-\theta D}] = \frac{\mu - \lambda}{\mu - \lambda + \theta} $$  

Plugging this expression into the traffic equation,

$$ \lambda = \frac{\mu - \lambda + \theta}{\mu - \lambda} \cdot \lambda_{ext} < \mu $$  
Without feedback
----------------

$$ \lambda_{ext} = \lambda < \mu $$  

In [ ]:
T = 100000

In [ ]:
theta = 0.1
arrival_rate = 10
service_rate = 13

In [ ]:
using_feedback = True

In [ ]:
if using_feedback:
    flow_rate = (service_rate + arrival_rate - np.sqrt((service_rate + arrival_rate)**2 - 4 * arrival_rate * (service_rate + theta))) / 2
else:
    flow_rate = arrival_rate
print flow_rate

In [ ]:
if using_feedback:
    expected_recall_rate = (service_rate - flow_rate) / (service_rate - flow_rate + theta)
else:
    expected_recall_rate = 1
print expected_recall_rate

In [ ]:
assert service_rate > flow_rate / expected_recall_rate

In [ ]:
using_clocked_delays = True
using_fifo = True

In [ ]:
q = Queue() if using_fifo else LifoQueue()
exits = []
delays = []

In [ ]:
t = 0
for _ in xrange(T):
    t += np.random.exponential(1 / (arrival_rate + service_rate))
    if np.random.random() < arrival_rate / (arrival_rate + service_rate):
        q.put(t)
    elif not q.empty():
        if using_clocked_delays:
            delay = t - q.get()
        else:
            q.get()
            delay = np.random.exponential(1 / (service_rate - flow_rate))
        delays.append(delay)
        if np.random.random() < np.exp(-theta*delay):
            exits.append(t)
        elif using_feedback:
            q.put(t)

In [ ]:
plt.xlabel('Simulated Delay')
plt.ylabel('Frequency (Number of Interactions)')
plt.hist(delays, bins=20, linewidth=0, alpha=0.5)
plt.show()

In [ ]:
w = np.zeros(int(np.ceil(exits[-1])))
for x in exits:
    w[int(x)] += 1

In [ ]:
s = np.random.poisson(flow_rate * expected_recall_rate, 100000)

In [ ]:
plt.xlabel('Number of Exits in 1-Unit Window')
plt.ylabel('Frequency (Number of Windows)')
plt.hist(s, bins=20, alpha=0.5, linewidth=0, normed=True, label=r'Poisson($\lambda = %0.3f$)' % (flow_rate * expected_recall_rate))
plt.hist(w, bins=20, alpha=0.5, linewidth=0, normed=True, label='Simulated')
plt.legend(loc='best')
plt.show()

In [ ]:
ex = np.array(exits)
interarrival_times = ex[1:] - ex[:-1]

In [ ]:
s = np.random.exponential(1 / (flow_rate * expected_recall_rate), 100000)

In [ ]:
plt.xlabel('Inter-Arrival Time')
plt.ylabel('Frequency (Number of Exits)')
plt.hist(interarrival_times, bins=20, alpha=0.5, linewidth=0, normed=True, label=r'Exponential($\lambda = %0.3f$)' % (flow_rate * expected_recall_rate))
plt.hist(interarrival_times, bins=20, alpha=0.5, linewidth=0, normed=True, label='Simulated')
plt.legend(loc='best')
plt.show()

In [ ]:
scaling_factors = np.arange(1, 100, 2)

In [ ]:
simulated_exit_rates = []
for scaling_factor in scaling_factors:    
    q = Queue()
    exits = []
    
    t = 0
    for _ in xrange(T):
        t += np.random.exponential(1 / (scaling_factor * (arrival_rate + service_rate)))
        if np.random.random() < arrival_rate / (arrival_rate + service_rate):
            q.put(t)
        elif not q.empty():
            if np.random.random() < np.exp(-theta*scaling_factor*(t - q.get())):
                exits.append(t)
            elif using_feedback:
                q.put(t)
                
    simulated_exit_rates.append(len(exits) / (scaling_factor * t))

In [ ]:
plt.xlabel('Scaling Factor')
plt.ylabel('Exit Rate')
plt.plot(scaling_factors, simulated_exit_rates, label='Simulated')
plt.plot(scaling_factors, [flow_rate * expected_recall_rate] * len(scaling_factors), '--', label='Predicted')
plt.legend(loc='best')
plt.show()